In [51]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [52]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [53]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'PublicHAPI',
            'X-POU': 'TREAT',
            'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [54]:
client = setup_fhir_client()

search_results = client.search('Patient', {
    'family': 'Smith',
    'gender': 'male',
    'birthdate': '1965-09-01',
    'address-state': 'NC',
})
Pretty(json.dumps(search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "896c9bc5-0a37-4610-a779-7dd7e292243c",
    "meta": {
        "lastUpdated": "2024-01-16T19:10:40.027+00:00"
    },
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://hapi.fhir.org/baseR4/Patient?address-state=NC&birthdate=1965-09-01&family=Smith&gender=male"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/53254",
            "resource": {
                "resourceType": "Patient",
                "id": "53254",
                "meta": {
                    "versionId": "4",
                    "lastUpdated": "2023-09-29T00:45:23.054+00:00",
                    "source": "#yv8lN6mkhf7deTiS"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">James <b>SMIT

There should be exactly one `Patient` matching the search.

In [55]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

{
    "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/53254",
    "resource": {
        "resourceType": "Patient",
        "id": "53254",
        "meta": {
            "versionId": "4",
            "lastUpdated": "2023-09-29T00:45:23.054+00:00",
            "source": "#yv8lN6mkhf7deTiS"
        },
        "text": {
            "status": "generated",
            "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">James <b>SMITH </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Identifier</td><td>110107073916280</td></tr><tr><td>Address</td><td><span>1234 Six Forks </span><br/><span>Cary </span><span>NC </span><span>US </span></td></tr><tr><td>Date of birth</td><td><span>01 September 1965</span></td></tr></tbody></table></div>"
        },
        "identifier": [
            {
                "use": "usual",
                "system": "1.3.6.1.4.1.22812.3.99930.3",
                "value": "110107073916280"
            },

In [56]:
HTML(patient['resource']['text']['div'])

Identifier,110107073916280
Address,1234 Six Forks Cary NC US
Date of birth,01 September 1965


In [57]:
everything = client.everything('Patient', patient['resource']['id'], count=1000)
Pretty(json.dumps(everything, indent=4))

{
    "resourceType": "Bundle",
    "id": "edd126f1-2674-4090-a4a0-82ab107b4396",
    "meta": {
        "lastUpdated": "2024-01-16T19:10:42.840+00:00"
    },
    "type": "searchset",
    "total": 127,
    "link": [
        {
            "relation": "self",
            "url": "https://hapi.fhir.org/baseR4/Patient/53254/$everything?_count=1000"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/53254",
            "resource": {
                "resourceType": "Patient",
                "id": "53254",
                "meta": {
                    "versionId": "4",
                    "lastUpdated": "2023-09-29T00:45:23.054+00:00",
                    "source": "#yv8lN6mkhf7deTiS"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">James <b>SMITH </b></div><table class=\"hapi

`MedicationAdministration` should have the medications administered to this Patient.

In [58]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "3c6f2fe7-e604-4f51-92e8-446a90f1a55c",
    "meta": {
        "lastUpdated": "2024-01-16T19:10:44.338+00:00"
    },
    "type": "searchset",
    "total": 0,
    "link": [
        {
            "relation": "self",
            "url": "https://hapi.fhir.org/baseR4/MedicationAdministration?patient=53254"
        }
    ]
}

`Observation` should have lab test results, and pregnancy status.

In [62]:
observation_search_results = client.search('Observation', query_params={
    # 'patient': patient['resource']['id']
    'code': '76272004'
})

Pretty(json.dumps(observation_search_results, indent=4))        

{
    "resourceType": "Bundle",
    "id": "9a0b29d0-8be7-44db-81bd-b265faa852f3",
    "meta": {
        "lastUpdated": "2024-01-16T20:54:08.517+00:00"
    },
    "type": "searchset",
    "total": 0,
    "link": [
        {
            "relation": "self",
            "url": "https://hapi.fhir.org/baseR4/Observation?code=76272004"
        }
    ]
}

`Encounter` should have hospitalization status.

In [60]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "2c73b962-490d-4691-a822-7d3025b1be38",
    "meta": {
        "lastUpdated": "2024-01-16T19:10:46.340+00:00"
    },
    "type": "searchset",
    "total": 3,
    "link": [
        {
            "relation": "self",
            "url": "https://hapi.fhir.org/baseR4/Encounter?patient=53254"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Encounter/53234",
            "resource": {
                "resourceType": "Encounter",
                "id": "53234",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2019-11-01T18:21:32.312+00:00",
                    "source": "#jkcy2CmYvZIMfcnV"
                },
                "status": "unknown",
                "class": {
                    "system": "http://terminology.hl7.org/ValueSet/v3-ActEncounterCode",
                    "code": "AMB",
                    "display": "ambulat

In [61]:
client.close()